# 包的导入

In [36]:
#导入需要用到的包
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 数据集的导入与描述

In [37]:
#数据读取
buyer = pd.read_excel(r"C:\Users\Administrator\Desktop\实习\hw\work_hw\buyers.xlsx")
order = pd.read_excel(r"C:\Users\Administrator\Desktop\实习\hw\work_hw\order.xlsx")
# 去除列名两侧空白，防止列名匹配失败
buyer.columns = buyer.columns.str.strip()
order.columns = order.columns.str.strip()
print('buyer columns:', buyer.columns.tolist())
print('order columns:', order.columns.tolist())

buyer columns: ['买方代码', '订单序号', '收货人姓名', '客户姓名', '客户性别', '客户年龄', '客户职业', '客户国家']
order columns: ['订单编号', '订单金额（美元）', '下单时间', '支付时间', '海关申报价值', '预计重量（KG）', '海关申报重量（KG）']


In [38]:
print('Buyer数据集基本信息:')
buyer.info() #buyer数据集基本信息
print("------------------------------")
print('Order数据集基本信息:')
order.info() #order数据集基本信息

Buyer数据集基本信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   买方代码    677 non-null    object 
 1   订单序号    651 non-null    float64
 2   收货人姓名   677 non-null    object 
 3   客户姓名    677 non-null    object 
 4   客户性别    677 non-null    object 
 5   客户年龄    677 non-null    int64  
 6   客户职业    677 non-null    object 
 7   客户国家    677 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 42.4+ KB
------------------------------
Order数据集基本信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   订单编号        651 non-null    float64
 1   订单金额（美元）    674 non-null    float64
 2   下单时间        677 non-null    object 
 3   支付时间        677 non-null    object 
 4   海关申报价值      677 non-null    int64  
 5   预计重量（KG）

In [39]:
buyer.describe(), order.describe()

(               订单序号        客户年龄
 count  6.510000e+02  677.000000
 mean   3.898618e+14   38.183161
 std    2.923020e+14    9.919147
 min    3.088386e+13   20.000000
 25%    9.389964e+13   31.000000
 50%    5.062334e+14   36.000000
 75%    7.022501e+14   42.000000
 max    7.029787e+14   79.000000,
                订单编号     订单金额（美元）      海关申报价值    预计重量（KG）  海关申报重量（KG）
 count  6.510000e+02   674.000000  677.000000  677.000000  677.000000
 mean   3.898618e+14    39.797671    2.205318    0.129258    0.191356
 std    2.923020e+14    71.261967    0.668900    0.200632    0.042070
 min    3.088386e+13     0.000000    0.000000    0.000000    0.000000
 25%    9.389964e+13     8.600000    2.000000    0.000000    0.200000
 50%    5.062334e+14    15.395000    2.000000    0.000000    0.200000
 75%    7.022501e+14    58.490000    3.000000    0.200000    0.200000
 max    7.029787e+14  1245.660000    5.000000    2.200000    0.380000)

In [40]:
def missing_value(df):
    return df.isnull().sum() / len(df) * 100

In [41]:
def missing():
    #所以看来只有订单序号里面有缺失值
    print(missing_value(buyer))

    print("--------------------------")
    #查看order里面的缺失值占比
    print(missing_value(order))
missing()

买方代码     0.000000
订单序号     3.840473
收货人姓名    0.000000
客户姓名     0.000000
客户性别     0.000000
客户年龄     0.000000
客户职业     0.000000
客户国家     0.000000
dtype: float64
--------------------------
订单编号          3.840473
订单金额（美元）      0.443131
下单时间          0.000000
支付时间          0.000000
海关申报价值        0.000000
预计重量（KG）      0.000000
海关申报重量（KG）    0.000000
dtype: float64


由于订单编号无法生成所以只能去除

In [42]:
# 去除buyer和order里面的缺失值（subset 需为列表）
buyer = buyer.dropna(subset=['订单序号'])
order = order.dropna(subset=['订单编号'])
#重新查看missing
missing()

买方代码     0.0
订单序号     0.0
收货人姓名    0.0
客户姓名     0.0
客户性别     0.0
客户年龄     0.0
客户职业     0.0
客户国家     0.0
dtype: float64
--------------------------
订单编号          0.000000
订单金额（美元）      0.460829
下单时间          0.000000
支付时间          0.000000
海关申报价值        0.000000
预计重量（KG）      0.000000
海关申报重量（KG）    0.000000
dtype: float64


In [43]:
#对订单金额先尝试转为数值，再用众数检查填充缺失值
order['订单金额（美元）'] = pd.to_numeric(order['订单金额（美元）'], errors='coerce')
mode_value = order['订单金额（美元）'].mode()
mode_value

0    8.6
Name: 订单金额（美元）, dtype: float64

In [44]:
# 用众数填充；如果没有众数则退回到中位数
if len(mode_value) > 0:
    fill_value = mode_value.iloc[0]
else:
    fill_value = order['订单金额（美元）'].median()
order['订单金额（美元）'].fillna(fill_value, inplace=True)  # 将缺失值填充

C:\Users\Administrator\AppData\Local\Temp\ipykernel_39328\3446067566.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  order['订单金额（美元）'].fillna(fill_value, inplace=True)  # 将缺失值填充


OK, 缺失值的填充和删除完成

# 进行异常值检查

In [45]:
buyer.columns

Index(['买方代码', '订单序号', '收货人姓名', '客户姓名', '客户性别', '客户年龄', '客户职业', '客户国家'], dtype='object')

In [ ]:
def order_quartile(column):
    q1 = order[column].quantile(0.25)
    q3 = order[column].quantile(0.75)
    IOR = q3 - q1
    lower_bound = q1 - 1.5 * IOR
    upper_bound = q3 + 1.5 * IOR
    return [lower_bound, upper_bound, IOR]
def buyer_quartile(column):
    q1 = buyer[column].quantile(0.25)
    q3 = buyer[column].quantile(0.75)
    IOR = q3 - q1
    lower_bound = q1 - 1.5 * IOR
    upper_bound = q3 + 1.5 * IOR
    return [lower_bound, upper_bound, IOR]

In [48]:
#删除重复行
buyer = buyer.drop_duplicates()
order = order.drop_duplicates()

In [ ]:
order_outlier_condition = (order['订单金额（美元）'] < order_quartile('订单金额（美元）')[0]) | (order['订单金额（美元）'] > order_quartile('订单金额（美元）')[1])
order_outliers = order[order_outlier_condition]
order["订单金额（美元）"] = np.clip(order['订单金额（美元）'], a_min=order_quartile('订单金额（美元）')[0], a_max=order_quartile('订单金额（美元）')[1])
order_outliers

,订单编号,订单金额（美元）,下单时间,支付时间,海关申报价值,预计重量（KG）,海关申报重量（KG）
15,9.250755e+13,159.64,2018-06-08 09:43:41.0,2018-06-08 09:43:46.0,3,0.000,0.200
31,9.439531e+13,132.42,2018-09-06 04:29:22.0,2018-09-07 17:18:29.0,2,0.000,0.200
55,9.573563e+13,148.19,2018-10-25 14:26:18.0,2018-10-25 14:26:21.0,2,0.000,0.200
90,9.339470e+13,864.04,2018-08-01 00:56:32.0,2018-08-01 00:57:09.0,2,0.200,0.200
99,9.299065e+13,1245.66,2018-07-14 05:40:36.0,2018-07-14 05:40:39.0,0,0.000,0.000
122,5.062982e+14,139.11,2018-07-06 16:16:11.0,2018-07-06 16:16:12.0,3,0.000,0.200
181,9.439776e+13,204.32,2018-09-01 10:48:50.0,2018-09-01 10:48:53.0,2,0.000,0.200
203,9.326323e+13,140.36,2018-07-20 19:04:28.0,2018-07-20 19:04:41.0,2,0.200,0.200
207,7.029787e+14,137.71,2018-11-05 20:09:03.0,2018-11-05 20:09:40.0,2,0.000,0.200
210,9.542176e+13,190.89,2018-10-13 02:30:30.0,2018-10-13 02:30:51.0,2,0.000,0.200


In [ ]:
buyer_outlier_condition = (buyer['客户年龄'] < buyer_quartile('客户年龄')[0]) | (buyer['客户年龄'] > buyer_quartile('客户年龄')[1])
buyer_outliers = buyer[buyer_outlier_condition]
buyer['客户年龄'] = np.clip(buyer['客户年龄'], a_min=buyer_quartile('客户年龄')[0], a_max=buyer_quartile('客户年龄')[1])
buyer_outliers

,买方代码,订单序号,收货人姓名,客户姓名,客户性别,客户年龄,客户职业,客户国家
15,jp176759296uqkae,9.250755e+13,Toyoko Komiya,toyoko komiya,female,66,Psychologist,日本
29,ua1082344887,9.367324e+13,Zhelo Olga,olga zhelo,female,60,Flight_attendant,乌克兰
54,sg1102109660jxim,9.325744e+13,Chan Hwee Leng,Hwee Leng Law,male,68,Chemical_engineer,新加坡
56,uk232998612ndeae,9.427121e+13,Miss stella fox,Stella Fox,female,61,Industrial_Engineer,英国
67,ru1131889704qbzt,7.022694e+14,Kabashova Anastasia Andreevna,Anastasia Kabashova,female,72,Economist,俄罗斯
69,us108372351gfuae,9.237527e+13,Elena Almeida,elena almeida,female,65,Physician,美国
82,ru146676668gppae,9.348432e+13,Denisova Lidiya Vladimirovna,LidiyaVladimirovna Denisova,male,61,Hotel_Manager,俄罗斯
136,kz1120880398cadw,9.494651e+13,Ishankulova Aigul,Aigul Ishankulova,female,64,Financial_Analyst,哈萨克斯坦
148,me1142077114jkcu,9.279113e+13,Tatjana Vujovic,Vujovic Tatjana,male,61,Consultant,黑山
192,ru1023047260,7.022208e+14,Postupaeva Marina Sergeevna,Marina Postupaeva,female,65,Software_Developer,俄罗斯
